In [4]:
from __future__ import division
from __future__ import print_function

import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from utils import load_data, accuracy
from gat_models import GAT

In [21]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [22]:
model = GAT(
    nfeat=features.shape[1], 
    nhid=8, 
    nclass=int(labels.max()) + 1, 
    dropout=0.6, 
    nheads=8, 
    alpha=0.2
)

optimizer = optim.Adam(
    model.parameters(), 
    lr=0.005, 
    weight_decay=5e-4
)

In [23]:
model.cuda()
features = features.cuda()
adj = adj.cuda()
labels = labels.cuda()
idx_train = idx_train.cuda()
idx_val = idx_val.cuda()
idx_test = idx_test.cuda()

In [24]:
features, adj, labels = Variable(features), Variable(adj), Variable(labels)

In [25]:
fastmode = False

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()),
          'time: {:.4f}s'.format(time.time() - t))

    return loss_val.data.item()

In [26]:
def compute_test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data),
          "accuracy= {:.4f}".format(acc_test.data))

In [31]:
# Train model
def train_model(number_of_epochs):
    t_total = time.time()
    patience = 10
    loss_values = []
    bad_counter = 0
    best = number_of_epochs + 1
    best_epoch = 0
    for epoch in range(number_of_epochs):
        loss_values.append(train(epoch))

        torch.save(model.state_dict(), '{}.pkl'.format(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1

        if bad_counter == patience:
            break

        files = glob.glob('*.pkl')
        for file in files:
            epoch_nb = int(file.split('.')[0])
            if epoch_nb < best_epoch:
                os.remove(file)

    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb > best_epoch:
            os.remove(file)

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    return best_epoch

In [32]:
best_epoch = train_model(100)

Epoch: 0001 loss_train: 0.9962 acc_train: 0.7500 loss_val: 1.0150 acc_val: 0.8300 time: 0.6818s
Epoch: 0002 loss_train: 0.9584 acc_train: 0.7714 loss_val: 1.0100 acc_val: 0.8333 time: 0.6727s
Epoch: 0003 loss_train: 0.9578 acc_train: 0.7643 loss_val: 1.0050 acc_val: 0.8300 time: 0.6741s
Epoch: 0004 loss_train: 0.9613 acc_train: 0.7643 loss_val: 1.0001 acc_val: 0.8300 time: 0.6735s
Epoch: 0005 loss_train: 1.0359 acc_train: 0.7214 loss_val: 0.9954 acc_val: 0.8300 time: 0.6749s
Epoch: 0006 loss_train: 0.9658 acc_train: 0.7571 loss_val: 0.9906 acc_val: 0.8300 time: 0.6732s
Epoch: 0007 loss_train: 0.9397 acc_train: 0.7429 loss_val: 0.9858 acc_val: 0.8300 time: 0.6743s
Epoch: 0008 loss_train: 0.9492 acc_train: 0.7429 loss_val: 0.9810 acc_val: 0.8300 time: 0.6738s
Epoch: 0009 loss_train: 0.8806 acc_train: 0.8000 loss_val: 0.9764 acc_val: 0.8300 time: 0.6736s
Epoch: 0010 loss_train: 0.9341 acc_train: 0.7643 loss_val: 0.9721 acc_val: 0.8300 time: 0.6739s
Epoch: 0011 loss_train: 1.0123 acc_train

In [33]:
# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

# Testing
compute_test()

Loading 99th epoch
Test set results: loss= 0.8069 accuracy= 0.8350
